In [5]:
# Base libraries
import time
import datetime
import os

# Scientific librariesz
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

# Visual libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Helper libraries
import xlrd
import pprint
import base64

# Visual setup

# Pandas options
pd.options.display.max_columns = None

## **1. Limpieza De Datos**
---
1. Crear sección de librerias de python
1. Cargar los dataset en la carpeta `1 bronce`
1. Homologar campos en las 3 bases 2019-eficiencia, 2020-eficiencia, 2021, con los campos identificados en el diccionario de datos (identificar las columnas que coinciden en las ters bases)
2. Crear una llave, concatenando el numero de documento con el tipo de documento
3. Teniedo en cuenta la llave del anterior punto, filtrar datos unicos
4. Filtrar para no tener en cuenta los datos nulos  
  * Validar columnas con datos socioeconomicos con valor nulo, con los datos del SISBEN (no tener en cuenta para el 26 de mayo)
5. Crear columna de año den cada base y unirlas (dataset final de trabajo) 

Despues de analizar el diccionario del Anexo 6A del año 2021, se seleccionaron las variables mas relevantes para el desarrollo del proyecto, para ello se creo una columna con el nonbre `COLUMNAS A TENER EN CUENTA` con los valores `SI`, `NO`, `SI SIN VALIDAR SED`, `SI PREGUNTAR SED`.

In [8]:
dic_anexo64_2021 =  pd.read_excel('../Diccionario_Anexo6A_31_03_2021_Modificado.xlsx', header=4, index_col=0)
dic_anexo64_2021

,NOMBRE VARIABLE,COLUMNAS A TENER EN CUENTA,ETIQUETA,TIPO DE VARIABLE,LONGITUD,MEDIDA,VALIDACIONES BÁSICAS O VALORES DE LA VARIABLE,VALIDACIONES,DESCRIPCION,OBSERVACIONES
No,,,,,,,,,,
1,ANO_INF,SI,Año de la información,Numérico,4,Nominal,Año al que corresponde la información,NaN,AÑO DE LA INFORMACIÓN,NaN
2,CODIGO_MUN,NO,Municipio o Distrito,Cadena,5,Nominal,Códigos DANE Municipios,"Debe estar acorde con la divipola, se toma el ...",CODIGO DANE DEL MUNICIPIO,"Corresponde al código 001, que en DIVIPOLA cor..."
3,CODIGO_DANE,SI,Código DANE del establecimiento,Numérico,12,Nominal,Código DANE de la institución educativa (12 po...,"Debe estar registrado en el DUE \nen estado ""N...",CODIGO DANE 12 DEL ESTABLECIMIENTO SEDE EDUCATIVA,NaN
4,CODIGO_DANE_SEDE,SI,Código DANE de la Sede,Numérico,12,Nominal,Código DANE que poseía la sede en el año 2001 ...,NaN,CODIGO DANE 12 DE LA SEDE EDUCATIVA,"Debe estar registrado en el DUE \nen estado ""A..."
5,CONS_SEDE,NO,Consecutivo de la sede,Numérico,14,Nominal,Código generado por le DUE para identificar lo...,NaN,CONSECUTIVO DE LA SEDE,NaN
...,...,...,...,...,...,...,...,...,...,...
93,DIR_SECTOR_CENSAL,NaN,Sector censal,Cadena,10,Nominal,NaN,Información del directorio con fecha de corte ...,Sector censal en el directorio,NaN
94,DIR_X,SI,Longitud,Numérico,20,Escala,NaN,Información del directorio con fecha de corte ...,Coordenada X del establecimiento en el directorio,NaN
95,DIR_Y,SI,Latitud,Numérico,20,Escala,NaN,Información del directorio con fecha de corte ...,Coordenada Y del establecimiento en el directorio,NaN


In [9]:
dic_anexo64_2021['COLUMNAS A TENER EN CUENTA'] = dic_anexo64_2021['COLUMNAS A TENER EN CUENTA'].replace(np.nan, 'NO')
main_variables = dic_anexo64_2021[dic_anexo64_2021['COLUMNAS A TENER EN CUENTA'].str.contains('SI', case=False)]['NOMBRE VARIABLE']
main_variables = main_variables.str.strip().tolist()
main_variables

['ANO_INF',
 'CODIGO_DANE',
 'CODIGO_DANE_SEDE',
 'TIPO_DOCUMENTO',
 'NRO_DOCUMENTO',
 'APELLIDO1',
 'APELLIDO2',
 'NOMBRE1',
 'NOMBRE2',
 'DIRECCION_RESIDENCIA',
 'RES_DEPTO',
 'RES_MUN',
 'ESTRATO',
 'SISBEN',
 'FECHA_NACIMIENTO',
 'GENERO',
 'POB_VICT_CONF',
 'DPTO_EXP',
 'MUN_EXP',
 'PROVIENE_SECTOR_PRIV',
 'PROVIENE_OTR_MUN',
 'TIPO_DISCAPACIDAD',
 'CAP_EXC',
 'CODIGO_ETNIA',
 'CODIGO_RESGUARDO',
 'INS_FAMILIAR',
 'CODIGO_JORNADA',
 'CARACTER',
 'CODIGO_ESPECIALIDAD',
 'CODIGO_GRADO',
 'CODIGO_METODOLOGIA',
 'REPITENTE',
 'SIT_ACAD_ANIO_ANT',
 'CON_ALUM_ANIO_ANT',
 'ZONA_RESI_ALU',
 'MADR_CABE_FAMI',
 'HIJO_MADR_CABE_FAMI',
 'BENE_VETE_FUER_PUBL',
 'HERO_NACI',
 'VALORACION1',
 'VALORACION2',
 'CODIGO_INTERNADO',
 'EDAD',
 'NIVEL',
 'NIVEL_B',
 'ETNIA_RECODIFICADA',
 'RANGO_EDAD',
 'CLASE_COLEGIO',
 'PER_ID',
 'PAIS_ORIGEN',
 'NOMBRE_PAIS_ORIGEN',
 'TIPO_NACIONALIDAD',
 'DIR_NUM_LOCALIDAD',
 'DIR_LOCALIDAD',
 'DIR_X',
 'DIR_Y',
 'VICTIMAS_INCLUIDO',
 'VICTIMAS_HECHO']

Se cargan los dataset para cada uno de los años

In [4]:
df_anexo6A_2017 = pd.read_csv('data/1-bronce/SIMAT_EFICIENCIA_2017.csv', delimiter=';')
df_anexo6A_2018 = pd.read_csv('data/1-bronce/SIMAT_EFICIENCIA_2018.csv', delimiter=';')
df_anexo6A_2019 = pd.read_csv('data/1-bronce/Anexo6A_Eficiencia_interna_MEN_2019.csv', delimiter=';')
df_anexo6A_2020 = pd.read_csv('data/1-bronce/Anexo6A_Eficiencia_interna_MEN_2020.csv', delimiter=';')
df_anexo6A_2021 = pd.read_csv('data/1-bronce/Anexo6A_depurado_31032021.csv', delimiter=';')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6,15,17,20,21,23,24,25,28,29,30,32,33,34,35,36,37,41,42,43,44,49,52,53,54,57,60,61,62,63,64,65,66,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,5,7,8,15,16,17,20,23,24,25,28,29,30,31,32,34,35,36,37,41,43,44,49,50,51,52,53,54,56,58,60,61,62,63,64,65,66,67,73,75,83,84,85,86,87,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36,49,71,74,76,77,89,92,95,104,105,106,108,115,116) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nod

Debido a que no en todos los dataset las columnas coinciden, primero miramos cuales son los nombers de columnas que coinciden

In [5]:
dfs = [ df_anexo6A_2017, df_anexo6A_2018, df_anexo6A_2019, df_anexo6A_2020, df_anexo6A_2021]
dfs_columns = np.concatenate([df.columns.str.strip() for df in dfs])
dfs_columns = pd.Series(dfs_columns).value_counts()
columns_in_all_dfs = dfs_columns[dfs_columns == len(dfs)]
columns_in_all_dfs = columns_in_all_dfs.index
columns_in_all_dfs

Index(['NIVEL', 'EXP_MUN', 'RES_DEPTO', 'DPTO_EXP', 'NRO_DOCUMENTO',
       'TIPO_DOCUMENTO', 'NOMBRE2', 'RES_MUN', 'NUEVO', 'CONS_SEDE', 'SISBEN',
       'APELLIDO2', 'TIPO_DISCAPACIDAD', 'MUN_EXP', 'NAC_DEPTO',
       'POB_VICT_CONF', 'CAP_EXC', 'FECHA_NACIMIENTO', 'EXP_DEPTO', 'EDAD',
       'REPITENTE', 'DIRECCION_RESIDENCIA', 'PROVIENE_SECTOR_PRIV',
       'APELLIDO1', 'GRUPO', 'TEL', 'CARACTER', 'NAC_MUN', 'ESTRATO',
       'NOMBRE1'],
      dtype='object')

In [6]:
missing_main_variables = list(filter(lambda x: x not in columns_in_all_dfs, main_variables))
missing_main_variables

['ANO_INF',
 'CODIGO_DANE',
 'CODIGO_DANE_SEDE',
 'GENERO',
 'PROVIENE_OTR_MUN',
 'CODIGO_ETNIA',
 'CODIGO_RESGUARDO',
 'INS_FAMILIAR',
 'CODIGO_JORNADA',
 'CODIGO_ESPECIALIDAD',
 'CODIGO_GRADO',
 'CODIGO_METODOLOGIA',
 'SIT_ACAD_ANIO_ANT',
 'CON_ALUM_ANIO_ANT',
 'ZONA_RESI_ALU',
 'MADR_CABE_FAMI',
 'HIJO_MADR_CABE_FAMI',
 'BENE_VETE_FUER_PUBL',
 'HERO_NACI',
 'VALORACION1',
 'VALORACION2',
 'CODIGO_INTERNADO',
 'NIVEL_B',
 'ETNIA_RECODIFICADA',
 'RANGO_EDAD',
 'CLASE_COLEGIO',
 'PER_ID',
 'PAIS_ORIGEN',
 'NOMBRE_PAIS_ORIGEN',
 'TIPO_NACIONALIDAD',
 'DIR_NUM_LOCALIDAD',
 'DIR_LOCALIDAD',
 'DIR_X',
 'DIR_Y',
 'VICTIMAS_INCLUIDO',
 'VICTIMAS_HECHO']

Debido a que se esta utilizando una gran cantidad de columnas, se necesita una funcion que encuntre coincidencias en los nombres de las otras columnas en los otros dataframes, con respecto a las columnas que definimos como importantes

In [7]:
def word_in_dfs_columns(dfs_dic, word, to_ignore=[]):
    words = word.split('_')
    words = list(filter(lambda x: x not in to_ignore, words))
    results = {}
    for key, df in dfs_dic.items():
        matches = []
        for w in words:
            matches = [*matches, *df.columns[df.columns.str.contains(w, case=False)]]
            matches = list(set(matches))
        
        results[key] = matches
    return results

In [8]:
df_dict = {'df_anexo6A_2017':df_anexo6A_2017,'df_anexo6A_2018':df_anexo6A_2018,'df_anexo6A_2019':df_anexo6A_2019, 'df_anexo6A_2020':df_anexo6A_2020}
variables = {}
for var in missing_main_variables:
    variables[var] = word_in_dfs_columns(df_dict, var, ['CODIGO', 'DIR', 'TIPO', 'B', 'ID'])
pprint.pprint(variables)

{'ANO_INF': {'df_anexo6A_2017': ['ANNO_INF',
                                 'CON_ALUM_ANO_ANT',
                                 'SIT_ACAD_ANO_ANT'],
             'df_anexo6A_2018': ['ANNO_INF',
                                 'CON_ALUM_ANO_ANT',
                                 'SIT_ACAD_ANO_ANT'],
             'df_anexo6A_2019': ['ANNO_INF',
                                 'CON_ALUM_ANO_ANT',
                                 'SIT_ACAD_ANO_ANT'],
             'df_anexo6A_2020': ['ANNO_INF',
                                 'CON_ALUM_ANO_ANT',
                                 'SIT_ACAD_ANO_ANT']},
 'BENE_VETE_FUER_PUBL': {'df_anexo6A_2017': [],
                         'df_anexo6A_2018': [],
                         'df_anexo6A_2019': [],
                         'df_anexo6A_2020': []},
 'CLASE_COLEGIO': {'df_anexo6A_2017': ['CLASE_COLEGIO_DIR', 'CLASE_SECTOR'],
                   'df_anexo6A_2018': ['CLASE_COLEGIO_DIR'],
                   'df_anexo6A_2019': ['RECTORCOLEGIOSDISTRI

In [9]:
word_in_dfs_columns(df_dict, 'LOC', ['CODIGO', 'DIR', 'TIPO', 'B', 'ID'])

{'df_anexo6A_2017': ['NUMERO_LOCALIDAD', 'NOMBRE_LOCALIDAD'],
 'df_anexo6A_2018': ['NUMERO_LOCALIDAD', 'NOMBRE_LOCALIDAD'],
 'df_anexo6A_2019': ['NOMBRE_LOCALIDAD', '@#_LOC'],
 'df_anexo6A_2020': ['NOMBRE_LOCALIDAD_DIR', 'LOC_DIR']}

Despues de analizar el resultado de nuestra funcion que encuentra coincidencias, creamos el sigueinte dicionario replace_columns_dict con los nombres de columnas a remplazr en los otros datasets

In [10]:

replace_columns_dict = {
    'ANNO_INF': 'ANO_INF',
    'DANE12_ESTABLECIMIENTO_EDUCATIVO': 'CODIGO_DANE',
    'DANE12_SEDE_EDUCATIVA': 'CODIGO_DANE_SEDE',
    'ESPECIALIDAD': 'CODIGO_ESPECIALIDAD',
    'ETNIA': 'CODIGO_ETNIA',
    'GRADO': 'CODIGO_GRADO',
    'INTERNADO': 'CODIGO_INTERNADO',
    'TIPO_JORNADA': 'CODIGO_JORNADA',
    'METODOLOGIA': 'CODIGO_METODOLOGIA',
    'CON_ALUM_ANO_ANT': 'CON_ALUM_ANIO_ANT',
    'GENERO.x': 'GENERO',
    'PROVIENE_OTRO_MUN': 'PROVIENE_OTR_MUN',
    'SIT_ACAD_ANO_ANT': 'SIT_ACAD_ANIO_ANT',
    'VAL_DES_PERIODO1': 'VALORACION1',
    'VAL_DES_PERIODO2': 'VALORACION2',
    'NUMERO_LOCALIDAD': 'DIR_NUM_LOCALIDAD',
    '@_LOC': 'DIR_NUM_LOCALIDAD',
    '@#_LOC': 'DIR_NUM_LOCALIDAD',
    'LOC_DIR': 'DIR_NUM_LOCALIDAD',
    'LOCALIDAD': 'DIR_LOCALIDAD',
    'NOMBRE_LOCALIDAD': 'DIR_LOCALIDAD',
    'NOMBRE_LOCALIDAD_DIR': 'DIR_LOCALIDAD',
    'ZON_ALU': 'ZONA_RESI_ALU',
    'NIVEL':'NIVEL'
}

La siguiente celda muestra como se relacionan las cada una de las coincidiencias definidas arriba, una pequeña muestra de los valores que tienen y el tipo
```
<replace_columns_dict_key> <replace_columns_dict_value>--> [values examples of dataframe 1] [values examples of dataframe 2] [values examples of dataframe 3]
types: [type of dataframe 1, type of dataframe 2, type of dataframe 3] 
```

In [11]:
text =""
for key, val in replace_columns_dict.items():
    text += f"{key}  {val}--> "
    var_types = []
    for df in dfs:
        
        if key in df.columns:
            text += f"{df[key].unique()[:5]}"
            var_types.append(df[key].dtype)
        elif val in df.columns:
            text += f"{df[val].unique()[:5]}"
            var_types.append(df[val].dtype)
        else:
            text += "(not found)"
            var_types.append("(not found)")
    text += f"\ntypes: {var_types} \n\n"   
print(text)

ANNO_INF  ANO_INF--> [2017 '2017' ' 146' ' 136' ' 133'][2018 '2018' '  83' '   0' '   1'][2019][2020][2021]
types: [dtype('O'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64')] 

DANE12_ESTABLECIMIENTO_EDUCATIVO  CODIGO_DANE--> [111001000078 111001000124 111001000132 111001000272 111001000353][111001000078 111001000124 111001000132 111001014176 111001000272][211850001473 311001090793 111001035572 111001012475 111001029955][111001000078 111001000124 111001000132 111001000272 111001000612][111001000078 111001000124 111001000132 111001000272 111001000612]
types: [dtype('int64'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64')] 

DANE12_SEDE_EDUCATIVA  CODIGO_DANE_SEDE--> [1.11001000e+11 1.11001015e+11 1.11001015e+11 1.11001000e+11
 1.11001014e+11][111001000078 111001000124 111001000132 111001000213 111001000272][211850000817 211001030630 311001090793 111001035572 111001013625][111001000078 111001014834 111001014842 111001000124 111001014303][111001000078 111001014834

Filtramos las variables que no se pudieron encontrar en todos los dataframes, incluso despues de hacer el checkeo manual.

In [12]:
unfindables_variables = list(filter(lambda x: x not in replace_columns_dict.values(), missing_main_variables))
unfindables_variables

['CODIGO_RESGUARDO',
 'INS_FAMILIAR',
 'MADR_CABE_FAMI',
 'HIJO_MADR_CABE_FAMI',
 'BENE_VETE_FUER_PUBL',
 'HERO_NACI',
 'NIVEL_B',
 'ETNIA_RECODIFICADA',
 'RANGO_EDAD',
 'CLASE_COLEGIO',
 'PER_ID',
 'PAIS_ORIGEN',
 'NOMBRE_PAIS_ORIGEN',
 'TIPO_NACIONALIDAD',
 'DIR_X',
 'DIR_Y',
 'VICTIMAS_INCLUIDO',
 'VICTIMAS_HECHO']

Ahora filtramos las variables que  vamos a utilizar y las asignamos a una lista en la variable `validate_main_variables` 

In [13]:
validate_main_variables = list(filter(lambda x: x not in unfindables_variables, main_variables))
validate_main_variables

['ANO_INF',
 'CODIGO_DANE',
 'CODIGO_DANE_SEDE',
 'TIPO_DOCUMENTO',
 'NRO_DOCUMENTO',
 'APELLIDO1',
 'APELLIDO2',
 'NOMBRE1',
 'NOMBRE2',
 'DIRECCION_RESIDENCIA',
 'RES_DEPTO',
 'RES_MUN',
 'ESTRATO',
 'SISBEN',
 'FECHA_NACIMIENTO',
 'GENERO',
 'POB_VICT_CONF',
 'DPTO_EXP',
 'MUN_EXP',
 'PROVIENE_SECTOR_PRIV',
 'PROVIENE_OTR_MUN',
 'TIPO_DISCAPACIDAD',
 'CAP_EXC',
 'CODIGO_ETNIA',
 'CODIGO_JORNADA',
 'CARACTER',
 'CODIGO_ESPECIALIDAD',
 'CODIGO_GRADO',
 'CODIGO_METODOLOGIA',
 'REPITENTE',
 'SIT_ACAD_ANIO_ANT',
 'CON_ALUM_ANIO_ANT',
 'ZONA_RESI_ALU',
 'VALORACION1',
 'VALORACION2',
 'CODIGO_INTERNADO',
 'EDAD',
 'NIVEL',
 'DIR_NUM_LOCALIDAD',
 'DIR_LOCALIDAD']

In [14]:
len(validate_main_variables)

40

Remplazamos los nombres de las columnas que encontramos pero que no coinciden en el resto de los dataframes.

In [15]:
for df in dfs:
    df.rename(columns=replace_columns_dict, inplace=True)

Despues de remplazar los nombres de las columnas, tomamos solos las que necesitamos 

In [16]:
df_anexo6A_2017 = df_anexo6A_2017[validate_main_variables]
df_anexo6A_2018 = df_anexo6A_2018[validate_main_variables]
df_anexo6A_2019 = df_anexo6A_2019[validate_main_variables]
df_anexo6A_2020 = df_anexo6A_2020[validate_main_variables]
df_anexo6A_2021 = df_anexo6A_2021[validate_main_variables]

In [17]:
df_anexo6A_2017['ANO_INF'] = '2017'
df_anexo6A_2018['ANO_INF'] = '2018'
df_anexo6A_2019['ANO_INF'] = '2019'
df_anexo6A_2020['ANO_INF'] = '2020'
df_anexo6A_2021['ANO_INF'] = '2021'

<ipython-input-17-5a990eaa5507>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2017['ANO_INF'] = '2017'
<ipython-input-17-5a990eaa5507>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2018['ANO_INF'] = '2018'
<ipython-input-17-5a990eaa5507>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

Igual que como se hizo anteriormente miramos las variables, una pequeña muestra de los valores que tienen y el tipo de estas
```
<dataframe_column> --> [values examples of dataframe 1] [values examples of dataframe 2] [values examples of dataframe 3]
types: [type of dataframe column 1, type of dataframe column 2, type of dataframe column 3] 
```

In [18]:
text =""
for col in validate_main_variables:
    text += f"{col} --> "
    var_types = []
    for df in dfs:
        text += f"{df[col].unique()[:5]}"
        var_types.append(df[col].dtype)
    text += f"\ntypes: {var_types} \n\n"   
print(text)

ANO_INF --> [2017 '2017' ' 146' ' 136' ' 133'][2018 '2018' '  83' '   0' '   1'][2019][2020][2021]
types: [dtype('O'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64')] 

CODIGO_DANE --> [111001000078 111001000124 111001000132 111001000272 111001000353][111001000078 111001000124 111001000132 111001014176 111001000272][211850001473 311001090793 111001035572 111001012475 111001029955][111001000078 111001000124 111001000132 111001000272 111001000612][111001000078 111001000124 111001000132 111001000272 111001000612]
types: [dtype('int64'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64')] 

CODIGO_DANE_SEDE --> [1.11001000e+11 1.11001015e+11 1.11001015e+11 1.11001000e+11
 1.11001014e+11][111001000078 111001000124 111001000132 111001000213 111001000272][211850000817 211001030630 311001090793 111001035572 111001013625][111001000078 111001014834 111001014842 111001000124 111001014303][111001000078 111001014834 111001014842 111001000124 111001000132]
types: [dtype('float64'

Checkeamos si existen duplicados en los dataframe, por ejemplo para número de documento

In [19]:
df_anexo6A_2017['CODIGO_DANE_SEDE'] = df_anexo6A_2017['CODIGO_DANE_SEDE'].astype(int)
df_anexo6A_2017['CODIGO_DANE_SEDE']

<ipython-input-19-f55e74703ee9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2017['CODIGO_DANE_SEDE'] = df_anexo6A_2017['CODIGO_DANE_SEDE'].astype(int)


0          111001000078
1          111001014834
2          111001014834
3          111001014834
4          111001014834
               ...     
1334072               0
1334073               0
1334074               0
1334075               0
1334076               0
Name: CODIGO_DANE_SEDE, Length: 1334077, dtype: int64

In [20]:
df_anexo6A_2017[df_anexo6A_2017['CODIGO_DANE_SEDE'] == 0]['CODIGO_DANE_SEDE'].value_counts()

0    143850
Name: CODIGO_DANE_SEDE, dtype: int64

In [21]:
df_anexo6A_2017.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
df_anexo6A_2018.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
df_anexo6A_2019.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
df_anexo6A_2020.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
df_anexo6A_2021.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)

<ipython-input-21-9d54c440ff9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2017.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
<ipython-input-21-9d54c440ff9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2018.drop_duplicates(subset=['NRO_DOCUMENTO'], keep=False, inplace=True)
<ipython-input-21-9d54c440ff9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anexo6A_2019.drop_duplica

In [22]:
df_anexo6A = pd.concat([df_anexo6A_2017, df_anexo6A_2018, df_anexo6A_2019, df_anexo6A_2020, df_anexo6A_2021], ignore_index=True)
df_anexo6A

,ANO_INF,CODIGO_DANE,CODIGO_DANE_SEDE,TIPO_DOCUMENTO,NRO_DOCUMENTO,APELLIDO1,APELLIDO2,NOMBRE1,NOMBRE2,DIRECCION_RESIDENCIA,RES_DEPTO,RES_MUN,ESTRATO,SISBEN,FECHA_NACIMIENTO,GENERO,POB_VICT_CONF,DPTO_EXP,MUN_EXP,PROVIENE_SECTOR_PRIV,PROVIENE_OTR_MUN,TIPO_DISCAPACIDAD,CAP_EXC,CODIGO_ETNIA,CODIGO_JORNADA,CARACTER,CODIGO_ESPECIALIDAD,CODIGO_GRADO,CODIGO_METODOLOGIA,REPITENTE,SIT_ACAD_ANIO_ANT,CON_ALUM_ANIO_ANT,ZONA_RESI_ALU,VALORACION1,VALORACION2,CODIGO_INTERNADO,EDAD,NIVEL,DIR_NUM_LOCALIDAD,DIR_LOCALIDAD
0,2017,111001000078,111001000078,2,00010803853,CRUZ,CANO,LAURA,VALENTINA,CL 69 A 105 F 67,11,1,2,",01",1/8/2000,F,9,,,N,N,99,9,0,6,1,5,10,1,N,1,9,1,,,3,17,4,16,PUENTE ARANDA
1,2017,111001000078,111001014834,3,05057637,NAVAS,GONZALEZ,SUJEIBY,VALENTINA,CL 34 SUR 40 A 51,11,1,3,,6/13/2012,F,9,,,N,N,99,9,0,3,0,0,0,1,N,0,9,1,,,3,4,1,16,PUENTE ARANDA
2,2017,111001000078,111001014834,3,067289989,RODRIGUEZ,GONZALEZ,SANTIAGO,JOSE,KR 54 SUR 50 B 10,11,1,3,,10/7/2008,M,9,,,N,N,7,9,0,3,0,0,3,1,N,1,9,1,,,3,8,2,16,PUENTE ARANDA
3,2017,111001000078,111001014834,3,071759788,AQUILES,,HERNANDEZ,MILANO,KR 40 28 A 02 SUR,11,1,3,,8/23/2012,M,9,,,N,N,99,9,0,3,0,0,0,1,N,0,9,1,,,3,4,1,16,PUENTE ARANDA
4,2017,111001000078,111001014834,3,089980482,ESPIN,CARDOZO,STEFANI,ALEJANDRA,KR 40 28 A 02 SUR,11,1,3,,11/16/2013,F,9,,,N,N,99,9,0,3,0,0,-1,1,N,0,9,1,,,,3,1,16,PUENTE ARANDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464566,2021,411102000293,411102000293,2,1012336508,REBELLON,TORRES,DAYANA,SULEY,KR 105 B 65 81 SUR,11,1,2,"54,68",9/6/2005,F,99,,,N,N,99,9,0,6,2,7,10,1,N,1,9,1,,,3,15,4,7,7
4464567,2021,411102000293,411102000293,2,1016713910,CARRILLO,TORRES,ASLHEY,DAYANNA,KR 99 A 71 39 SUR,11,1,2,"61,21",10/1/2005,F,99,,,N,N,99,9,0,6,2,7,10,1,N,1,9,1,,,3,15,4,7,7
4464568,2021,411102000293,411102000293,2,1011090034,PEREZ,CUERVO,KAROLL,MICHELLE,CL 62 SUR 93 C 53,11,1,2,"58,21",11/30/2005,F,99,,,N,N,99,9,0,6,2,7,10,1,N,1,9,1,,,3,15,4,7,7
4464569,2021,411102000293,411102000293,2,1011092068,MORALES,RODRIGUEZ,VALERIA,,CL 61 B SUR 81 D 03,11,1,2,"58,24",2/24/2006,F,99,,,N,N,99,9,0,6,2,7,10,1,N,1,9,1,,,3,15,4,7,7


In [23]:
for col in df_anexo6A.columns:
    text =""
    text += f"{col} --> "    
    text += f"{df_anexo6A[col].unique()[:]}"
    var_type = df_anexo6A[col].dtype
    text += f"\ntypes: {var_type} \n\n"   
    print(text)

ANO_INF --> ['2017' '2018' '2019' '2020' '2021']
types: object 


CODIGO_DANE --> [111001000078 111001000124 111001000132 ... 111001800678 111001800694
 111001800813]
types: object 


CODIGO_DANE_SEDE --> [111001000078 111001014834 111001014842 ... 111102000672 111001800813
 211850001309]
types: object 


TIPO_DOCUMENTO --> [2 3 5 7 1 6 8 9 '2' '7' '5' '1' '6' '*' '0' 0 10 11 12 ' 5' ' 3' ' 7'
 ' 2' ' 1' ' 6' '**']
types: object 


NRO_DOCUMENTO --> ['00010803853' '05057637' '067289989' ... '1215713431' '1065822914'
 '1028781395']
types: object 


APELLIDO1 --> ['CRUZ' 'NAVAS' 'RODRIGUEZ' ... 'MACCHIARULLO' 'BOYS' 'LENGUERKE']
types: object 


APELLIDO2 --> ['CANO' 'GONZALEZ' ' ' ... 'CIVIRA' 'MENESESS' 'FERRONEZ']
types: object 


NOMBRE1 --> ['LAURA' 'SUJEIBY' 'SANTIAGO' ... 'YORGEIDYS' 'BRYANLLERLYS' 'WITNNY']
types: object 


NOMBRE2 --> ['VALENTINA' 'JOSE' 'MILANO' ... 'SHEDYEL' 'MARAIRE' 'JESSIMAR']
types: object 


DIRECCION_RESIDENCIA --> ['CL 69 A 105 F 67' 'CL 34 SUR 40 A 51'

 campos ' ': `SISBEN`, `DPTO_EXP, MUN_EXP`, `VALORACION1`, `VALORACION2`, `CODIGO_INTERNADO`, `PAIS_ORIGEN`  
campos '\$null\$': `NOMBRE_PAIS_ORIGEN` 

In [24]:
empty_columns = ['SISBEN', 'DPTO_EXP', 'MUN_EXP', 'VALORACION1', 'VALORACION2', 'CODIGO_INTERNADO']

In [25]:
for col in empty_columns:
  print(f"Empties in {col} --> {df_anexo6A[df_anexo6A[col] == ' '][col].value_counts()[0]}")

Empties in SISBEN --> 1814713
Empties in DPTO_EXP --> 4271842
Empties in MUN_EXP --> 4271842
Empties in VALORACION1 --> 4401025
Empties in VALORACION2 --> 4402929
Empties in CODIGO_INTERNADO --> 572358


`DPTO_EXP` 2156194 vacios pero no todos los estudiantes son expulsados como victimas de conflicto  
`MUN_EXP ` 2156194 vacios pero no todos los estudiantes son expulsados como victimas de conflicto
`VALORACION1` y `VALORACION2` contiene muchos vacios, pero despues de ralizar un conteo en la base del 2019 donde se encuentran 27099 repitentes, solo 4 cuentan con el dato de valoracion, en este caso no tendriamos en cuneta estas variables para el análisis

In [26]:
df_anexo6A.to_csv('data/1-bronce/Consolidado-SIMAT-2017-2021.csv')